In [9]:
import numpy as np
import pandas as pd

import lightgbm
import matplotlib.pylab as plt

In [10]:
feature_dsb_train = pd.read_csv("features_train_csv/distance_store_busstop.csv")
feature_aipg_train = pd.read_csv("features_train_csv/average_income_per_grunnkrets.csv")
feature_nag_train = pd.read_csv("features_train_csv/number_adults_grunnkrets.csv")


In [11]:
features = []
features.append(feature_dsb_train)
features.append(feature_aipg_train)
features.append(feature_nag_train)

In [18]:
features_csv = features[0]
features_csv
features[1]
#features_csv = features_csv.merge(features[1], how="left")
#for feature in features:
    

,distance_store_busstop
0,585.354489
1,138.798366
2,41.099519
3,56.693017
4,229.077959
...,...
12854,150.004125
12855,322.573926
12856,174.851316
12857,121.372663
